# 3pt data exploration

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from copy import deepcopy

In [20]:
# sys.path.append(r"/Users/fordfishman/GitHub/3pt-shooting/code/python/")
sys.path.append(r"/home/fordfishman/GitHub/3pt-shooting/code/python/")
from basketball_reference_scraper.teams import get_roster_stats



In [21]:
year_range = list( range(2010, 2022) )
team_abr = [
    'ATL',
    'BRK',
    'BOS',
    'CHA',
    'CHI',
    'CLE',
    'DAL',
    'DEN',
    'DET',
    'GSW',
    'HOU',
    'IND',
    'LAC',
    'LAL',
    'MEM',
    'MIA',
    'MIL',
    'MIN',
    'NOP',
    'NYK',
    'OKC',
    'ORL',
    'PHI',
    'PHO',
    'POR',
    'SAC',
    'SAS',
    'TOR',
    'UTA',
    'WAS',
    ]




In [22]:
def player_index(names, seasons):

    indices = list()

    for i in range(len(names)):

        name = names[i].replace(' ', '')
        indices.append( '%s%s' % (name, seasons[i]) )

    return indices
    
def get_data(mode):

    df = get_roster_stats('GSW', 2021, mode)
    df = pd.DataFrame().reindex(columns=df.columns)

    for year in year_range:
        for team in team_abr:

            df_i = get_roster_stats(team, year, mode)
            df_i['index'] = player_index(df_i.PLAYER, df_i.SEASON)
            df = df.append(df_i, ignore_index=True)

    return df


In [23]:
# df_adv = get_data('ADVANCED')
# df_tot = get_data('TOTALS')
# df_poss = get_data('PER_POSS')
# df_adv.to_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv", index=False)
# df_poss.to_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv", index=False)
# df_tot.to_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv", index=False)

In [24]:
df_poss = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv")
df_poss.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Unnamed: 29',
       'ORtg', 'DRtg', 'SEASON', 'index'],
      dtype='object')

have to get combine player data from in-season trades
then prune data for attempts
then prune for consecutive seasons

In [25]:
df_adv = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv")
df_adv.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'SEASON', 'index'],
      dtype='object')

In [26]:
df_tot = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv")
df_tot.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Unnamed: 29',
       'ORtg', 'DRtg', 'SEASON', 'eFG%', 'index'],
      dtype='object')

In [27]:
df_tot = df_tot.drop(['Unnamed: 29'], axis=1)
df_poss = df_poss.drop(['Unnamed: 29'], axis=1)
df_adv = df_adv.drop(['Unnamed: 19', 'Unnamed: 24'], axis=1)

remove duplicate columns, print out duplicates

In [79]:
for col in df_tot:
    
    if col in df_poss and df_tot[col].equals(df_poss[col]):

        df_poss = df_poss.drop([col], axis=1)
        print(col)
        
    if col in df_adv and df_tot[col].equals(df_adv[col]):
        
        df_adv = df_adv.drop([col], axis=1)
        print(col)


key
key


rename possession columns

In [66]:
df_poss = df_poss.add_suffix('_poss')
# df_poss = df_poss.rename({"index_poss":"index"}, axis="columns")
# df_poss = df_poss.rename({"index":"key"}, axis="columns")
# df_adv = df_adv.rename({"index":"key"}, axis="columns")
# df_tot = df_tot.rename({"index":"key"}, axis="columns")


combine data frames

In [83]:
df_totposs = df_tot.join(df_poss, how='outer')
df_all = df_totposs.join(df_adv, how='outer')
df_all.columns


Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg', 'DRtg',
       'SEASON', 'eFG%', 'key', 'FG_poss', 'FGA_poss', '3P_poss', '3PA_poss',
       '2P_poss', '2PA_poss', 'FT_poss', 'FTA_poss', 'ORB_poss', 'DRB_poss',
       'TRB_poss', 'AST_poss', 'STL_poss', 'BLK_poss', 'TOV_poss', 'PF_poss',
       'PTS_poss', 'ORtg_poss', 'DRtg_poss', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [103]:
df_trade = deepcopy(df_all)

df_trade['trade'] = 0

idents = dict() # dictionary of id to list of indexes

for i, row in df_all.iterrows():

    ident = row.key

    if not str(row.key) in idents:
        idents[ident] = [i]

    else:
        idents[ident].append(i)

for ident, inds in idents.items():

    if len(inds) > 1:
        for i in inds:
            df_trade.trade[i] = 1




/tmp/ipykernel_10380/4174378321.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade.trade[i] = 1


1624

need to fix so that each player year is a single row

In [94]:
a = [0, 2,3]
b = [1,2,3,4,5,6,6]
b[slice(a)] 

TypeError: slice indices must be integers or None or have an __index__ method